## The Well rSVD Preprocessing

[link to official documentation](https://polymathic-ai.org/the_well/tutorials/dataset/)  
[link to original paper](https://proceedings.neurips.cc/paper_files/paper/2024/file/4f9a5acd91ac76569f2fe291b1f4772b-Paper-Datasets_and_Benchmarks_Track.pdf)

## Imports

In [1]:

import pprint as pp
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import torch
from einops import rearrange
from tqdm import tqdm

from the_well.benchmark.metrics import VRMSE
from the_well.data import WellDataset
from the_well.utils.download import well_download
from the_well.data import WellDataset
from torch.utils.data import DataLoader

debug = True
n_rank = 50
n_iters = 5
dataset = 'planetswe'
n_steps = {'planetswe': 1008, 'active_matter': 81}

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:

mae = lambda datatrue, datapred: (datatrue - datapred).abs().mean()
mse = lambda datatrue, datapred: (datatrue - datapred).pow(2).sum(axis = -1).mean()
mre = lambda datatrue, datapred: ((datatrue - datapred).pow(2).sum(axis = -1).sqrt() / (datatrue).pow(2).sum(axis = -1).sqrt()).mean()
num2p = lambda prob : ("%.2f" % (100*prob)) + "%"

def min_max_scale(tensor, feature_range=(0, 1)):
    """
    Scale a tensor to a given feature range using min-max normalization.
    
    Args:
        tensor (torch.Tensor): Input tensor to be scaled
        feature_range (tuple): Desired range of transformed data (default: (0, 1))
        
    Returns:
        torch.Tensor: Scaled tensor
        tuple: (min, max) values used for scaling (for inverse transformation)
    """
    # Ensure the input is a tensor
    if not isinstance(tensor, torch.Tensor):
        tensor = torch.tensor(tensor, dtype=torch.float32)
    
    # Calculate min and max
    t_min = tensor.min()
    t_max = tensor.max()
    
    # Avoid division by zero
    t_range = t_max - t_min
    if t_range == 0:  # all values are the same
        t_range = 1
    
    # Scale to [0, 1] first
    scaled = (tensor - t_min) / t_range
    
    # Then scale to feature_range
    min_range, max_range = feature_range
    scaled = scaled * (max_range - min_range) + min_range
    
    return scaled, (t_min, t_max)

def inverse_min_max_scale(scaled_tensor, original_min_max, feature_range=(0, 1)):
    """
    Inverse transformation of min-max scaling.
    
    Args:
        scaled_tensor (torch.Tensor): Scaled tensor to transform back
        original_min_max (tuple): (min, max) values from original scaling
        feature_range (tuple): Range used in original scaling (default: (0, 1))
        
    Returns:
        torch.Tensor: Tensor in original scale
    """
    t_min, t_max = original_min_max
    min_range, max_range = feature_range
    
    # First scale back to [0, 1] range
    normalized = (scaled_tensor - min_range) / (max_range - min_range)
    
    # Then scale back to original range
    original = normalized * (t_max - t_min) + t_min
    
    return original

def create_mats(the_well_data, combine_all=False, debug=False):
    im_shape = the_well_data[0]["input_fields"].shape
    n_steps, im_rows, im_cols, im_dim = im_shape[0], im_shape[1], im_shape[2], im_shape[3]

    mats = []
    for i in range(len(the_well_data)):
        data = rearrange(the_well_data[i]["input_fields"], "t r c d -> t (r c d)", t=n_steps, r=im_rows, c=im_cols, d=im_dim)
        mats.append(data)
        if debug:
            break
    if combine_all:
        mats = [torch.cat(mats, dim=0)]
    return mats

def generate_SVDs(mats, n_rank=50, n_iters=2):
    Us, Ss, Vs = ([], [], [])
    for mat in mats:
        U, S, V = torch.svd_lowrank(mat, n_rank, n_iters)
        Us.append(U)
        Ss.append(S)
        Vs.append(V)
    return Us, Ss, Vs

def create_pods(mats, V_mats):
    pods = []
    for mat, V in zip(mats, V_mats):
        pod = mat @ V
        pods.append(pod)
    return pods

def scale_pods(pods):
    pods_scaled = []
    scalers_l = []
    for pod in pods:
        pod_scaled, scalers = min_max_scale(pod)
        pods_scaled.append(pod_scaled)
        scalers_l.append(scalers)
    return pods_scaled, scalers_l

def inverse_pod(pods_scaled, scalers_l, V_mats):
    mats_hat = []
    for pod_scaled, scalers, V_mat in zip(pods_scaled, scalers_l, V_mats):
        mat_hat = inverse_min_max_scale(pod_scaled, scalers)
        mat_hat = mat_hat @ V_mat.T
        mats_hat.append(mat_hat)
    return mats_hat


## Load Data

In [3]:

# Load data from online, when using in practice we'll have to download the dataset
train_data = WellDataset(
    well_base_path=Path('/data') / 'alexey' / 'the_well',
    well_dataset_name=dataset,
    well_split_name="train",
    n_steps_input=n_steps[dataset],
    n_steps_output=0,
    use_normalization=False,
)

valid_data = WellDataset(
    well_base_path=Path('/data') / 'alexey' / 'the_well',
    well_dataset_name=dataset,
    well_split_name="valid",
    n_steps_input=n_steps[dataset],
    n_steps_output=0,
    use_normalization=False,
)

test_data = WellDataset(
    well_base_path=Path('/data') / 'alexey' / 'the_well',
    well_dataset_name=dataset,
    well_split_name="test",
    n_steps_input=n_steps[dataset],
    n_steps_output=0,
    use_normalization=False,
)




## Concatenate

In [4]:

combine_all = True
train_mats = create_mats(train_data, combine_all=combine_all, debug=debug)
valid_mats = create_mats(valid_data, combine_all=combine_all, debug=debug)
test_mats = create_mats(test_data, combine_all=combine_all, debug=debug)

print(train_mats[0].shape)
print(valid_mats[0].shape)
print(test_mats[0].shape)


torch.Size([1008, 393216])
torch.Size([1008, 393216])
torch.Size([1008, 393216])


## POD

In [5]:

U_trains, S_trains, V_trains = generate_SVDs(train_mats, n_rank=n_rank, n_iters=n_iters)
U_valids, S_valids, V_valids = generate_SVDs(valid_mats, n_rank=n_rank, n_iters=n_iters)
U_tests, S_tests, V_tests = generate_SVDs(test_mats, n_rank=n_rank, n_iters=n_iters)


In [6]:

train_pods = create_pods(train_mats, V_trains)
valid_pods = create_pods(valid_mats, V_valids)
test_pods = create_pods(test_mats, V_tests)

print(train_pods[0].shape)


torch.Size([1008, 50])


In [7]:

train_pods_scaled, train_scalers = scale_pods(train_pods)
valid_pods_scaled, valid_scalers = scale_pods(valid_pods)
test_pods_scaled, test_scalers = scale_pods(test_pods)


## Inverse POD

In [8]:

train_mats_hat = inverse_pod(train_pods_scaled, train_scalers, V_trains)
valid_mats_hat = inverse_pod(valid_pods_scaled, valid_scalers, V_valids)
test_mats_hat = inverse_pod(test_pods_scaled, test_scalers, V_tests)


## Print Errors

In [9]:

def print_errors(true_l, pred_l, error_f, title):
    print(title)
    for i, (true, pred) in enumerate(zip(true_l, pred_l)):
        print(f"Error for i={i} is {num2p(error_f(true, pred))}")
    print()

print_errors(train_mats, train_mats_hat, mre, "Training POD errors:")
print_errors(valid_mats, valid_mats_hat, mre, "Validation POD errors:")
print_errors(test_mats, test_mats_hat, mre, "Testing POD errors:")


Training POD errors:
Error for i=0 is 1.34%

Validation POD errors:
Error for i=0 is 1.45%

Testing POD errors:
Error for i=0 is 1.18%

